Split Folder

In [43]:
import os
import shutil
import random
from collections import defaultdict

DATASET_DIR = "./dataset"
OUTPUT_DIR = "./splitted_dataset"
RATIOS = (0.7, 0.15, 0.15)
SEED = 1337

random.seed(SEED)

def get_patient_id(fname):
    return "_".join(fname.split("_")[:2])  # OAS1_0028_MR1

# --------------------------------------------------
# 1. Kumpulkan file per (class, patient)
# --------------------------------------------------
patients_per_class = defaultdict(lambda: defaultdict(list))

for cls in os.listdir(DATASET_DIR):
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    for fname in os.listdir(cls_path):
        pid = get_patient_id(fname)
        patients_per_class[cls][pid].append(fname)

# --------------------------------------------------
# 2. Split patient secara stratified per class
# --------------------------------------------------
splits = {"train": [], "val": [], "test": []}

for cls, patient_dict in patients_per_class.items():
    patient_ids = list(patient_dict.keys())
    random.shuffle(patient_ids)

    n = len(patient_ids)

    if n < 2:
        print(f"[WARNING] Class '{cls}' hanya punya {n} patient. Tidak bisa split masuk akal.")
        splits["train"].extend([(cls, pid) for pid in patient_ids])
        continue

    n_train = max(1, int(RATIOS[0] * n))
    n_val   = max(1, int(RATIOS[1] * n)) if n >= 3 else 0

    if n_train + n_val >= n:
        n_train = n - 1
        n_val = 0

    train_ids = patient_ids[:n_train]
    val_ids   = patient_ids[n_train:n_train + n_val]
    test_ids  = patient_ids[n_train + n_val:]

    splits["train"].extend([(cls, pid) for pid in train_ids])
    splits["val"].extend([(cls, pid) for pid in val_ids])
    splits["test"].extend([(cls, pid) for pid in test_ids])

# --------------------------------------------------
# 3. Copy atau move file
# --------------------------------------------------
for split, items in splits.items():
    for cls, pid in items:
        out_dir = os.path.join(OUTPUT_DIR, split, cls)
        os.makedirs(out_dir, exist_ok=True)

        for fname in patients_per_class[cls][pid]:
            src = os.path.join(DATASET_DIR, cls, fname)
            dst = os.path.join(out_dir, fname)
            shutil.copy2(src, dst)  # ganti ke move() kalau mau cepat


# Importing & Configuration

In [44]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score
import copy
import timm
from tqdm import tqdm
from ultralytics import YOLO

In [45]:
DATASET_DIR = "./splitted_dataset"
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCH = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Data Transformers

In [46]:
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

# Data Loading

In [47]:
image_datasets = {x: datasets.ImageFolder(f"{DATASET_DIR}/{x}", data_transforms[x]) for x in ['train','val']}
dataloaders = {x: DataLoader(image_datasets[x],batch_size=BATCH_SIZE,shuffle=True, num_workers=2) for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes

print(class_names)
print(dataset_sizes)

['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
{'train': 60573, 'val': 12261}


# Preprocessing

In [48]:
y_train = image_datasets['train'].targets
class_counts = np.bincount(y_train)
total_samples = len(y_train)
class_weights = total_samples / (len(class_names) * class_counts)
class_weights = torch.FloatTensor(class_weights).to(DEVICE)

# Training

Training Function

In [49]:
def train_model(model:nn.Module,model_name:str,criterion:nn.CrossEntropyLoss, optimizer:torch.optim.Optimizer, num_epochs = EPOCH):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    history = {'train_loss':[],'val_f1':[]}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print("-"*10)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            all_preds = []
            all_labels = []

            pbar = tqdm(dataloaders[phase], desc=f'Epoch {epoch+1} {phase}')

            for inputs, labels in pbar:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                pbar.set_postfix({'loss':loss.item()})
                running_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim = 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = f1_score(all_labels,all_preds,average='macro')

            print(f'{phase} Loss: {epoch_loss:.4f} F1: {epoch_f1:.4f}')

            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(),f'best_models/{model_name}_best.pth')
                print(f'New best model is saved with F1 : {best_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model


In [50]:
model_to_test = ['resnetv2_50.a1h_in1k', 'convnext_tiny','xception']

In [ ]:
for model_name in model_to_test:
    print(f"{'='*10}")
    print(f"{model_name}'s Training")
    print(f'{'='*10}')

    model = timm.create_model(model_name,pretrained=True,num_classes = NUM_CLASSES)
    model = model.to(DEVICE)

    criteration = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)
    train_model(model,model_name,criteration,optimizer,num_epochs=EPOCH)

resnetv2_50.a1h_in1k's Training
Epoch 1/20
----------


Epoch 1 train: 100%|██████████| 1893/1893 [04:54<00:00,  6.43it/s, loss=0.503]


train Loss: 0.9214 F1: 0.4735


Epoch 1 val: 100%|██████████| 384/384 [01:34<00:00,  4.04it/s, loss=5.43]


val Loss: 5.9803 F1: 0.0994
New best model is saved with F1 : 0.0994
Epoch 2/20
----------


...

Epoch 20 train: 100%|██████████| 1893/1893 [04:41<00:00,  6.72it/s, loss=0.000159]


train Loss: 0.0076 F1: 0.9960


Epoch 20 val: 100%|██████████| 384/384 [00:30<00:00, 12.57it/s, loss=24.4]


val Loss: 23.3647 F1: 0.1092
convnext_tiny's Training
Epoch 1/20
----------


Epoch 1 train:  46%|████▌     | 869/1893 [04:38<05:17,  3.22it/s, loss=2.09] 

In [ ]:
yolo_model = torch.hub.load('ultralytics/yolov5','yolov5s',pretrained = True)
train_model(model,'YOLO',criteration,optimizer,num_epochs=EPOCH)


Using cache found in C:\Users\Alvin/.cache\torch\hub\ultralytics_yolov5_master


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...

   ---------------------------------------- 0/3 [smmap]
   ------------- -------------------------- 1/3 [gitdb]
   ------------- -------------------------- 1/3 [gitdb]
   ------------- -------------------------- 1/3 [gitdb]
   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   ---------------------------------------- 3/3 [gitpython]


requirements: AutoUpdate success  4.9s
WARNING requirements: Restart runtime or rerun command for updates to take effect



YOLOv5  2025-12-18 Python-3.13.5 torch-2.10.0.dev20251211+cu130 CUDA:0 (NVIDIA GeForce RTX 5060 Laptop GPU, 8151MiB)

100%|██████████| 14.1M/14.1M [00:01<00:00, 10.4MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Epoch 1/20
----------


Epoch 1 train:  19%|█▉        | 357/1891 [01:07<04:52,  5.25it/s, loss=0.00545] 


KeyboardInterrupt: 

# Evaluate

In [ ]:
def evaluate_on_test_set(model:nn.Module,model_name):
    model.load_state_dict(torch.load(f'./best_models/{model_name}_best.pth'))
    model.eval()

    all_preds = []
    all_labels = []

    test_dataset = datasets.ImageFolder(f'{DATASET_DIR}/test',data_transforms['val'])
    test_loader = DataLoader(test_dataset,batch_size =BATCH_SIZE,shuffle=False)

    with torch.no_grad():
        for inputs,labels in test_loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            preds = torch.argmax(outputs,dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
    print(classification_report(all_labels,all_preds, target_names=test_dataset.classes))

In [ ]:
for model_name in model_to_test:
    evaluate_on_test_set(model,model_name)

                    precision    recall  f1-score   support

     Mild Dementia       1.00      1.00      1.00       751
 Moderate Dementia       1.00      1.00      1.00        74
      Non Demented       1.00      1.00      1.00     10084
Very mild Dementia       1.00      1.00      1.00      2060

          accuracy                           1.00     12969
         macro avg       1.00      1.00      1.00     12969
      weighted avg       1.00      1.00      1.00     12969



FileNotFoundError: [Errno 2] No such file or directory: './best_models/convnext_tiny_best.pth'

In [ ]:
evaluate_on_test_set(yolo_model,'YOLO')